### **프로젝트 개요**
이 프로젝트는 한국어 법령 질의회신 텍스트에서 '질의 ID', '답변 내용' 등 특정 정보를 자동으로 추출하는 개체명 인식(Named Entity Recognition, NER) 모델을 구축하는 과정을 다룹니다. `klue/bert-base` 모델을 기반으로 Hugging Face 라이브러리를 사용하여 모델을 파인튜닝합니다.

### **학습 목표**
1.  **데이터 준비**: 원시 텍스트를 정제하고, 라벨링 도구(Doccano)에 적합한 형태로 가공합니다.
2.  **데이터 라벨링**: Doccano를 사용하여 텍스트에 개체명 태그를 지정합니다.
3.  **모델 학습**: 라벨링된 데이터를 BERT 모델 학습에 맞게 변환하고, Hugging Face `Trainer`를 사용하여 모델을 파인튜닝합니다.
4.  **모델 평가 및 추론**: 학습된 모델의 성능을 평가하고, 새로운 텍스트에서 개체명을 추출하는 방법을 확인합니다.

---

## **1. 환경 설정**
필요한 라이브러리를 설치하고 Google Drive를 연동하여 프로젝트 환경을 구성합니다.

In [ ]:
# 1.1. 필요한 라이브러리 설치
# Hugging Face 생태계의 핵심 라이브러리와 성능 평가 도구를 설치합니다.
!pip install transformers[torch] datasets evaluate seqeval accelerate -q

In [ ]:
# 1.2. 라이브러리 임포트
import os
import json
import re
import numpy as np
import torch
import pandas as pd
from google.colab import drive
from datasets import Dataset, Features, Value, ClassLabel, Sequence
from transformers import AutoTokenizer, AutoModelForTokenClassification, TrainingArguments, Trainer, DataCollatorForTokenClassification
import evaluate # 최신 성능 평가 라이브러리

# 1.3. Google Drive 마운트
# Colab 환경에서 Google Drive에 접근하기 위해 마운트합니다.
drive.mount('/content/gdrive')

# 1.4. 프로젝트 경로 설정
# Google Drive 내에 프로젝트 폴더를 설정하고, 데이터와 모델을 저장할 하위 폴더를 생성합니다.
# **사용자 환경에 맞게 이 경로를 수정해주세요.**
project_root = '/content/gdrive/MyDrive/Colab Notebooks/'
project_dir = os.path.join(project_root, 'deep-learning-ner') # 프로젝트 디렉토리 경로 (본인의 프로젝트 디렉토리명으로 수정)
data_dir = os.path.join(project_dir, 'data')
model_dir = os.path.join(project_dir, 'model')

# 디렉토리 생성 (이미 존재하면 건너뜀)
os.makedirs(project_dir, exist_ok=True)
os.makedirs(data_dir, exist_ok=True)
os.makedirs(model_dir, exist_ok=True)

print(f"프로젝트 루트: {project_root}")
print(f"데이터 폴더: {data_dir}")
print(f"모델 저장 폴더: {model_dir}")

---

## **2. 데이터 준비 및 전처리**
딥러닝 모델의 성능은 데이터의 품질에 크게 좌우됩니다. 이 단계에서는 원시 데이터를 모델 학습에 적합한 형태로 가공합니다.

### **2.1. 원본 텍스트 전처리 (로컬 PC에서 수행 권장)**
라벨링 작업의 효율성을 높이기 위해, Doccano에 데이터를 임포트하기 전에 간단한 전처리를 수행합니다. 아래 파이썬 코드는 불필요한 문자열을 제거하고, 각 질의-회신 쌍을 별개의 문서로 분리하는 역할을 합니다.

**권장 사항:** 이 코드는 로컬 PC 환경(예: VSCode)에서 원본 `dataset.txt` 파일에 대해 실행한 후, 결과물인 `dataset_cleaned.txt` 파일을 Google Drive의 `data` 폴더에 업로드하세요.

In [ ]:
# 아래 코드는 로컬 PC에서 실행하기 위한 예시입니다. (Colab에서 실행 X)
# def clean_text_for_doccano(input_path, output_path):
#     """
#     Doccano 라벨링을 위해 원본 텍스트 파일을 전처리합니다.
#     - 특정 문자열 제거
#     - 정규표현식을 이용해 '질의 N.' 패턴을 기준으로 문서 분리 (개행 문자 2개 추가)
#     """
#     try:
#         with open(input_path, 'r', encoding='utf-8') as f:
#             content = f.read()
#
#         # 불필요한 문자열, 페이지 번호 등 제거 (예시)
#         content = re.sub(r'2021년 소방시설법령 질의회신집', '', content)
#         content = re.sub(r'\n\d{1,3}\n', '', content) # 페이지 번호 제거
#
#         # '질의'로 시작하는 부분을 찾아 두 번의 개행으로 분리 (Doccano에서 별도 문서로 인식)
#         # re.sub의 콜백 함수를 사용하여 첫 '질의'는 제외
#         def replace_marker(match):
#             if match.start() == 0:
#                 return match.group(0)
#             else:
#                 return '\n\n' + match.group(0)
#
#         processed_content = re.sub(r'질의\\s*\\d+\\.?', replace_marker, content.strip())
#
#         with open(output_path, 'w', encoding='utf-8') as f:
#             f.write(processed_content)
#
#         print(f"전처리 완료! '{output_path}' 파일이 생성되었습니다.")
#
#     except Exception as e:
#         print(f"오류 발생: {e}")
#
# # 로컬에서 사용할 경우
# # input_file = 'dataset.txt'
# # output_file = 'dataset_cleaned.txt'
# # clean_text_for_doccano(input_file, output_file)

### **2.2. Doccano를 이용한 데이터 라벨링**
1.  **Doccano 설치/실행**: 로컬 PC에 Docker를 설치한 후, 터미널에서 `docker run -it -p 8000:8000 doccano/doccano` 명령으로 Doccano를 실행합니다.
2.  **프로젝트 생성**: 웹 브라우저에서 `http://localhost:8000`에 접속하여 새 프로젝트를 만들고, 프로젝트 타입을 `Sequence Labeling`으로 선택합니다.
3.  **레이블 정의**: 인식할 개체명 레이블(예: `QUESTION_ID`, `ANSWER_CONTENT`, `LAW_NAME` 등)을 정의합니다.
4.  **데이터 임포트**: `data` 폴더에 업로드한 `dataset_cleaned.txt` 파일을 `Plain Text` 형식으로 임포트합니다.
5.  **라벨링 수행**: 텍스트를 드래그하여 해당하는 레이블을 지정합니다.
6.  **데이터 익스포트**: 라벨링이 완료되면 `Export Data` 탭에서 `JSONL` 형식으로 데이터를 다운로드합니다. **'Export only approved documents' 옵션을 반드시 체크**하여 검수가 끝난 데이터만 내보냅니다.
7.  **파일 업로드**: 다운로드한 `JSONL` 파일을 Google Drive의 `data` 폴더에 업로드합니다. (예: `after_datalabeling.jsonl`)

### **2.3. Colab에서 라벨링 데이터 로드 및 가공**
Doccano에서 익스포트한 `JSONL` 파일을 로드하고, `klue/bert-base` 토크나이저가 이해할 수 있는 형태로 변환합니다. 이 과정에서 각 토큰에 BIO (Beginning, Inside, Outside) 태그를 할당합니다.

In [ ]:
# 2.3.1. 라벨 및 토크나이저 설정
# Doccano에서 정의한 레이블 목록 (B-, I- 접두사 제외)
doccano_labels = [
    "QUESTION_ID", "QUESTION_CONTENT", "ANSWER_ID", 
    "ANSWER_CONTENT", "LAW_NAME", "MISC_HEADER"
]

# BIO 체계에 따라 전체 라벨 목록 생성
label_list = ["O"]
for label in doccano_labels:
    label_list.append(f"B-{label}")
    label_list.append(f"I-{label}")

# 라벨과 ID를 매핑하는 딕셔너리 생성
label_to_id = {label: i for i, label in enumerate(label_list)}
id_to_label = {i: label for i, label in enumerate(label_list)}
num_labels = len(label_list)

# klue/bert-base 토크나이저 로드
tokenizer = AutoTokenizer.from_pretrained("klue/bert-base")

print("전체 라벨 목록:", label_list)
print("\n라벨 -> ID 매핑:", label_to_id)


# 2.3.2. Doccano JSONL 파일 로드 및 Dataset 객체로 변환
# **사용자가 업로드한 실제 파일 이름으로 수정해주세요.**
labeled_data_filename = 'after_datalabeling.jsonl'
labeled_data_filepath = os.path.join(data_dir, labeled_data_filename)

try:
    with open(labeled_data_filepath, 'r', encoding='utf-8') as f:
        # 'label' 키를 'labels'로 변경하여 리스트에 저장
        doccano_data = []
        for line in f:
            item = json.loads(line)
            # Doccano 1.x는 'label'을, 최신 버전은 'labels'를 사용하므로 호환성 확보
            annotations = item.pop('label', item.pop('labels', []))
            item['labels'] = annotations
            doccano_data.append(item)
            
    # Hugging Face Dataset 객체로 변환
    raw_dataset = Dataset.from_list(doccano_data)
    print(f"\n로드된 라벨링 데이터 샘플 수: {len(raw_dataset)}")
    print("로드된 데이터 첫 번째 샘플:", raw_dataset[0])

except FileNotFoundError:
    print(f"'{labeled_data_filepath}' 파일을 찾을 수 없습니다. 파일 이름과 경로를 확인하세요.")
except Exception as e:
    print(f"파일 로드 중 오류 발생: {e}")

# 2.3.3. 토큰화 및 라벨 정렬 함수 정의
def tokenize_and_align_labels(examples):
    # is_split_into_words=False: 문장 전체를 입력으로 받음
    # return_offsets_mapping=True: 토큰과 원본 문자열의 위치 관계를 얻기 위해 필요
    tokenized_inputs = tokenizer(
        examples["text"], 
        truncation=True, 
        max_length=512, 
        return_offsets_mapping=True
    )

    aligned_labels = []
    for i, labels_in_doc in enumerate(examples["labels"]):
        offset_mapping = tokenized_inputs["offset_mapping"][i]
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        
        # 모든 토큰을 'O'로 초기화
        current_labels = np.full(len(offset_mapping), label_to_id['O'])

        # 라벨링된 엔티티 순회
        for start, end, label in labels_in_doc:
            # 해당 엔티티의 B- 태그와 I- 태그 ID
            b_label_id = label_to_id.get(f"B-{label}")
            i_label_id = label_to_id.get(f"I-{label}")

            if b_label_id is None or i_label_id is None:
                continue

            # 원본 텍스트의 start, end 오프셋에 해당하는 토큰 찾기
            token_indices = [
                idx for idx, (o_start, o_end) in enumerate(offset_mapping)
                if o_start >= start and o_end <= end
            ]
            
            if token_indices:
                # 첫 토큰에 B- 태그 할당
                current_labels[token_indices[0]] = b_label_id
                # 나머지 토큰에 I- 태그 할당
                for idx in token_indices[1:]:
                    current_labels[idx] = i_label_id

        # 특수 토큰([CLS], [SEP], [PAD])의 라벨을 -100으로 설정하여 손실 계산에서 제외
        final_labels = []
        for word_id, label_id in zip(word_ids, current_labels):
            final_labels.append(-100 if word_id is None else label_id)
        aligned_labels.append(final_labels)

    tokenized_inputs["labels"] = aligned_labels
    return tokenized_inputs

# 2.3.4. 데이터셋에 함수 적용 및 분할
if 'raw_dataset' in locals():
    processed_dataset = raw_dataset.map(
        tokenize_and_align_labels,
        batched=True,
        remove_columns=raw_dataset.column_names
    )

    # 학습용/평가용 데이터셋 분할 (80:20)
    train_test_split = processed_dataset.train_test_split(test_size=0.2, seed=42)
    train_dataset = train_test_split['train']
    eval_dataset = train_test_split['test']

    print("\n데이터 전처리 및 분할 완료:")
    print(f"학습 데이터셋 샘플 수: {len(train_dataset)}")
    print(f"평가 데이터셋 샘플 수: {len(eval_dataset)}")
    print("\n전처리된 학습 데이터 샘플:")
    print(train_dataset[0])

---
## **3. 모델 학습**
전처리가 완료된 데이터셋을 사용하여 `klue/bert-base` 모델을 파인튜닝합니다.

In [ ]:
# 3.1. 모델 로드
# 토큰 분류(NER)를 위한 BERT 모델을 로드합니다.
# id2label, label2id를 전달하여 모델이 라벨 정보를 알 수 있도록 합니다.
model = AutoModelForTokenClassification.from_pretrained(
    "klue/bert-base",
    num_labels=num_labels,
    id2label=id_to_label,
    label2id=label_to_id
)

print("모델 로드 완료.")
print("주의: 일부 가중치가 초기화되지 않았다는 경고는 정상입니다. NER 헤드(분류기)가 새로 추가되었기 때문이며, 이 부분은 파인튜닝을 통해 학습됩니다.")

# 3.2. 성능 지표(Metrics) 정의
# `evaluate` 라이브러리의 `seqeval` 모듈을 사용하여 F1, 정밀도, 재현율을 계산합니다.
seqeval_metric = evaluate.load("seqeval")

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # 예측과 실제 라벨에서 -100을 제외하고 실제 라벨 이름으로 변환
    true_predictions = [
        [id_to_label[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [id_to_label[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = seqeval_metric.compute(predictions=true_predictions, references=true_labels)
    
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

# 3.3. 학습 인자(Training Arguments) 설정
# GPU 메모리가 부족하면 `per_device_train_batch_size`를 8, 4 등으로 줄여보세요.
# 데이터가 적을 경우 `num_train_epochs`를 늘려볼 수 있으나, 과적합에 주의해야 합니다.
training_args = TrainingArguments(
    output_dir=model_dir,
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=5,  # 데이터 양에 따라 조절
    weight_decay=0.01,
    evaluation_strategy="epoch", # 매 에포크 종료 시 평가
    save_strategy="epoch",       # 매 에포크 종료 시 모델 저장
    load_best_model_at_end=True, # 학습 종료 후 최적 성능 모델 로드
    metric_for_best_model="f1",  # F1 점수를 기준으로 최적 모델 선택
    push_to_hub=False,
    report_to="none",
)

# 3.4. Trainer 설정 및 학습 시작
# DataCollator는 배치 내의 데이터를 동일한 길이로 패딩하는 역할을 합니다.
data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

print("\n모델 학습을 시작합니다...")
# **참고:** 데이터셋이 매우 작으면 (수십 ~ 수백 개), F1 점수가 0으로 나올 수 있습니다.
# 이는 모델이 'O' 태그만 예측하는 안전한 방향으로 학습하기 때문입니다.
# 더 많은 데이터를 라벨링하면 성능이 점차 향상됩니다.
trainer.train()

# 3.5. 학습된 최적 모델 저장
trainer.save_model(os.path.join(model_dir, "best_model"))
print(f"\n학습 완료! 최적 모델이 '{os.path.join(model_dir, 'best_model')}'에 저장되었습니다.")

---
## **4. 모델 추론 (Inference)**
학습된 모델을 사용하여 새로운 텍스트에서 개체명을 추출하는 테스트를 수행합니다.

In [ ]:
# 4.1. 저장된 모델 및 토크나이저 로드
# 가장 성능이 좋았던 모델을 로드합니다.
model_path = os.path.join(model_dir, "best_model")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

loaded_tokenizer = AutoTokenizer.from_pretrained(model_path)
loaded_model = AutoModelForTokenClassification.from_pretrained(model_path).to(device)

print(f"저장된 모델을 로드했습니다. ({model_path})")
print(f"추론은 '{device}' 장치에서 실행됩니다.")


# 4.2. 개체명 인식 추론 함수 정의
def predict_ner(text, tokenizer, model):
    model.eval() # 모델을 평가 모드로 설정
    
    # 텍스트 토큰화
    tokenized_input = tokenizer(
        text, 
        return_tensors="pt", 
        truncation=True, 
        max_length=512
    ).to(model.device)

    # 모델 추론
    with torch.no_grad():
        outputs = model(**tokenized_input)

    # 예측 결과(logits)에서 가장 확률이 높은 라벨 ID 추출
    predictions = torch.argmax(outputs.logits, dim=2)
    
    # 토큰과 예측된 라벨을 함께 순회하며 엔티티 재구성
    tokens = tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"][0])
    predicted_labels = [model.config.id2label[i] for i in predictions[0].cpu().numpy()]
    
    results = []
    current_entity_tokens = []
    current_entity_label = None

    for token, label in zip(tokens, predicted_labels):
        # [CLS], [SEP] 등 특수 토큰은 건너뜀
        if token in tokenizer.all_special_tokens:
            continue

        # B- 태그: 새로운 엔티티의 시작
        if label.startswith("B-"):
            # 이전에 수집 중이던 엔티티가 있다면 먼저 저장
            if current_entity_tokens:
                entity_text = tokenizer.convert_tokens_to_string(current_entity_tokens)
                results.append({"entity": entity_text, "label": current_entity_label})
            
            current_entity_tokens = [token]
            current_entity_label = label[2:] # 'B-' 접두사 제거
        
        # I- 태그: 현재 엔티티에 계속 속함
        elif label.startswith("I-"):
            # 이전 엔티티와 타입이 같은 경우에만 토큰 추가
            if current_entity_label == label[2:]:
                current_entity_tokens.append(token)
            # 타입이 다르면 이전 엔티티를 저장하고, 현재 토큰은 무시 (또는 새로운 B-로 처리)
            else:
                if current_entity_tokens:
                    entity_text = tokenizer.convert_tokens_to_string(current_entity_tokens)
                    results.append({"entity": entity_text, "label": current_entity_label})
                current_entity_tokens = []
                current_entity_label = None

        # O 태그: 엔티티가 끝남
        else:
            if current_entity_tokens:
                entity_text = tokenizer.convert_tokens_to_string(current_entity_tokens)
                results.append({"entity": entity_text, "label": current_entity_label})
            current_entity_tokens = []
            current_entity_label = None
            
    # 반복문 종료 후 마지막 엔티티가 남아있다면 추가
    if current_entity_tokens:
        entity_text = tokenizer.convert_tokens_to_string(current_entity_tokens)
        results.append({"entity": entity_text, "label": current_entity_label})
        
    return results

# 4.3. 모델 테스트
test_text = "질의 1 연면적 450㎡인 특정소방대상물에 최초 건축물 사용승인시에 비상경보설비 설치가 되지 않은 경우 건축허가일과 사용승인일 중 소방시설설치기준 적용일은? 회신 1 건축물 등의 신축ㆍ증축ㆍ개축ㆍ재축ㆍ이전ㆍ용도변경 또는 대수선의 허가 ㆍ협의 및 사용승인의 권한이 있는 행정기관은 소방시설법 제7조제1항에 따라 소재지를 관할하는 소방본부장이나 소방서장의 동의를 받아야 하므로, 건축허가등과 관련한 협의과정이 누락되었다면, 건축허가 신청일을 기준으로 소방시설의 설치기준을 적용합니다."

predicted_entities = predict_ner(test_text, loaded_tokenizer, loaded_model)

print("\n--- 추론 결과 ---")
print("입력 텍스트:", test_text)
print("\n예측된 개체명:")
# 결과를 보기 좋게 출력
for entity in predicted_entities:
    print(f"- {entity['entity']} ({entity['label']})")

---
## **5. 결론 및 회고**

### **성능 향상을 위한 다음 단계**
현재 모델은 소량의 데이터로 학습되어 성능이 만족스럽지 않을 수 있습니다. 모델 성능을 높이기 위한 가장 효과적인 방법은 다음과 같습니다.

1.  **더 많은 데이터 라벨링 (가장 중요)**: Doccano로 돌아가 최소 수백 개 이상의 다양한 예시를 라벨링합니다. 데이터의 양과 질이 모델 성능을 결정하는 핵심 요소입니다.
2.  **하이퍼파라미터 튜닝**: 충분한 데이터가 확보된 후, `TrainingArguments`의 `learning_rate`, `num_train_epochs` 등을 조정하여 성능을 최적화할 수 있습니다.
3.  **약 지도 학습(Weak Supervision) 활용**: 정규표현식(Regex) 등을 사용하여 '질의 1', '회신 1'과 같이 명확한 패턴을 가진 개체를 자동으로 라벨링하는 '약 지도 학습' 기법을 적용할 수 있습니다. 이렇게 생성된 데이터를 Doccano에서 검토하고 수정하면 라벨링 시간을 크게 단축할 수 있습니다.

### **프로젝트 회고**
* **End-to-End 파이프라인 구축**: 데이터 전처리, 라벨링, 모델 학습, 평가, 추론에 이르는 전체 과정을 직접 경험하며 머신러닝 프로젝트의 흐름을 실질적으로 이해했습니다.
* **데이터의 중요성**: "Garbage In, Garbage Out" 원칙을 체감했습니다. 모델의 성능은 복잡한 아키텍처보다 고품질의 학습 데이터에 더 크게 의존한다는 것을 확인했습니다.
* **실용적 자동화의 가치**: 비록 100% 완벽한 모델은 아니더라도, 반복적인 수작업을 자동화하여 업무 효율을 크게 향상시키는 것만으로도 프로젝트의 가치는 충분합니다. 완벽함보다 개선을 목표로 하는 접근 방식의 중요성을 깨달았습니다.